Main folder containing images is only relative to the computer where the training of the CNN will occur
For the the time being, it's Rostom's PC

In [16]:
#Imports
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import tensorflow_hub as hub
import matplotlib.pylab as plt

<h3>Setup</h3>

In [2]:
#Main folder containing images
main_folder = 'C:/Users/Rostom/Videos/5SDBD/Projet_Intégrateur/archive/'

#Images details
IMG_WIDTH = 178
IMG_HEIGHT = 218
#BATCH_SIZE = 16
#NUM_EPOCHS = 20

In [3]:
#Datagrame containing all images vectors
df_attr = pd.read_csv(main_folder + 'list_attr_celeba.csv')
#df_attr.set_index('image_id', inplace=True)
df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
df_attr.shape
df_attr

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
202595,202596.jpg,0,0,0,0,0,1,1,0,0,...,0,1,1,0,0,0,0,0,0,1
202596,202597.jpg,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
202597,202598.jpg,0,1,1,0,0,0,1,0,1,...,0,1,0,1,1,0,1,0,0,1


In [4]:
#Enumerate all features
for i, j in enumerate(df_attr.columns):
    print(i, j)

0 image_id
1 5_o_Clock_Shadow
2 Arched_Eyebrows
3 Attractive
4 Bags_Under_Eyes
5 Bald
6 Bangs
7 Big_Lips
8 Big_Nose
9 Black_Hair
10 Blond_Hair
11 Blurry
12 Brown_Hair
13 Bushy_Eyebrows
14 Chubby
15 Double_Chin
16 Eyeglasses
17 Goatee
18 Gray_Hair
19 Heavy_Makeup
20 High_Cheekbones
21 Male
22 Mouth_Slightly_Open
23 Mustache
24 Narrow_Eyes
25 No_Beard
26 Oval_Face
27 Pale_Skin
28 Pointy_Nose
29 Receding_Hairline
30 Rosy_Cheeks
31 Sideburns
32 Smiling
33 Straight_Hair
34 Wavy_Hair
35 Wearing_Earrings
36 Wearing_Hat
37 Wearing_Lipstick
38 Wearing_Necklace
39 Wearing_Necktie
40 Young


<h3>Splitting Data</h3>

In [5]:
#Dataframe for partitions (0 = train, 1 = validation, 2 = test)
df_partition = pd.read_csv(main_folder + "list_eval_partition.csv")
df_partition.sample(5)
#df_partition.set_index('image_id', inplace=True)
df_partition


,image_id,partition
0,000001.jpg,0
1,000002.jpg,0
2,000003.jpg,0
3,000004.jpg,0
4,000005.jpg,0
...,...,...
202594,202595.jpg,2
202595,202596.jpg,2
202596,202597.jpg,2
202597,202598.jpg,2


In [6]:
#Joining Partition df with main df
df_total = pd.merge(df_attr, df_partition, how="inner", on="image_id")
df_total.head(5)

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,partition
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,1,1,0,1,0,1,0,0,1,0
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,1,0
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0


In [7]:
#Sample sizes for training, validation and testing
TRAINING_SAMPLES = 10000
VALIDATION_SAMPLES = 2000
TEST_SAMPLES = 2000


In [18]:
#Splitting Data

train_df = df_total.loc[df_total["partition"] == 0]
train_df_test = train_df.head(TRAINING_SAMPLES)

val_df = df_total.loc[df_total["partition"] == 1]
val_df_test = val_df.head(VALIDATION_SAMPLES)

test_df = df_total.loc[df_total["partition"] == 2]
test_df_test = test_df.head(TEST_SAMPLES)



In [8]:
#Setting up the model
# Import InceptionV3 Model
inc_model = InceptionV3(weights='../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
                        include_top=False,
                        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

#inc_model.summary()

ValueError: The `weights` argument should be either `None` (random initialization), `imagenet` (pre-training on ImageNet), or the path to the weights file to be loaded; Received: weights=../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5